In [1]:
# SSPSR CHIKUSI NOVAL
import pdb
import math
import time  
import argparse
import json
import os
from os import listdir
from os.path import join
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import numpy as np
import scipy.io as scio
# from scipy.misc import imresize
from scipy.signal import convolve2d
from skimage.measure import compare_psnr, compare_ssim

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data as data
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchnet import meter


# from data_utils import TrainsetFromFolder, ValsetFromFolder
# from data_utils import is_image_file
# from data_utils2 import HSTrainingData, HSTestData
# from data_utils3 import loadingData,loadingTestData
from data_utils4 import TrainsetFromFolder, ValsetFromFolder
from data_utils4 import is_image_file

from eval import PSNR, SSIM, SAM
# from eval2 import quality_assessment

from loss import HybridLoss, SAMLoss
# from model.MCNet import MCNet
# from model.SSAC import SSAC
# from model.SSPSR import SSPSR
# from model.SSPSR import default_conv
from LGFFMN import LGFFMN

In [2]:
# Training settings
# without val
parser = argparse.ArgumentParser(description="Super-Resolution")

parser.add_argument("--model_title", type=str, default="WTSR00", help="model_title, default set to model_title")
# parser.add_argument('--method', default='ASFS', type=str, help='super resolution method name')

# Training settings
# parser.add_argument("--upscale_factor", default=4, type=int, help="super resolution upscale factor")
parser.add_argument("--upscale_factor", default=4, type=int, help="super resolution upscale factor")
parser.add_argument("--seed", type=int, default=3000, help="start seed for model")
parser.add_argument("--batchSize", type=int, default=32, help="training batch size")
# parser.add_argument("--batchSize", type=int, default=64, help="training batch size")
# parser.add_argument("--nEpochs", type=int, default=85, help="maximum number of epochs to train")
parser.add_argument("--nEpochs", type=int, default=200, help="maximum number of epochs to train")
# parser.add_argument("--show", action="store_true", help="show Tensorboard")
parser.add_argument("--show", type=int, default=1, help="show Tensorboard")
parser.add_argument("--lr", type=int, default=1e-4, help="lerning rate")
# parser.add_argument("--cuda", action="store_true", help="Use cuda")
parser.add_argument("--cuda", type=int, required=False,default=1, help="set it to 1 for running on GPU, 0 for CPU")
# parser.add_argument("--gpus", default="0,1,2,3", type=str, help="gpu ids (default: 0)")
parser.add_argument("--gpus", default="2,3", type=str, help="gpu ids (default: 0)")
parser.add_argument("--threads", type=int, default=12, help="number of threads for dataloader to use")
parser.add_argument("--resume", default="", type=str, help="Path to checkpoint (default: none)  checkpoint/model_epoch_95.pth")
parser.add_argument("--start-epoch", default=1, type=int, help="Manual epoch number (useful on restarts)")               
parser.add_argument("--datasetName", default="CAVE", type=str, help="data name")
parser.add_argument("--weight_decay", type=float, default=0, help="weight decay, default set to 0")



#SFCSR # Network settings

# parser.add_argument('--n_module', type=int, default=5, help='number of  modules')
# parser.add_argument('--n_feats', type=int, default=64, help='number of feature maps')



# Test image
# parser.add_argument('--model_name', default='checkpoint/CAVE_model_4_epoch_XX.pth', type=str, help='super resolution model name ')
parser.add_argument('--model_name', default='', type=str, help='super resolution model name ')


# opt = parser.parse_args() 
opt = parser.parse_args(args=[])

In [3]:
global writer
write_path ='/media/dy113/disk1/Project_xjr/hyperfusion/Mine/logs/'+ opt.model_title + '/' + opt.datasetName + '/' + str(opt.upscale_factor) + '/' + 'nfeats128/'
writer = SummaryWriter(write_path)
# writer = SummaryWriter('./logs/') 
# out_path = '/home/sunjianjun/liqiang/Dataset/test/' + opt.datasetName + '/' 
# out_path = '/media/dy113/disk1/Project_xjr/hyperfusion/Mine/result/' + opt.model_title + '/' + opt.datasetName + '/' + opt.upscale_factor + '/'
resume = True
# resume = False
log_interval = 50
psnr = []   

In [4]:
def main():

    # if opt.show:
    #     global writer
    #     writer = SummaryWriter(log_dir='logs') 
       
    if opt.cuda:
        print("=> Use GPU ID: '{}'".format(opt.gpus))
        os.environ["CUDA_VISIBLE_DEVICES"] = opt.gpus
        if not torch.cuda.is_available():
            raise Exception("No GPU found or Wrong gpu id, please run without --cuda")
		
    torch.manual_seed(opt.seed)
    if opt.cuda:
        torch.cuda.manual_seed(opt.seed)
    cudnn.benchmark = True
    
   # Loading datasets
    # train_set = TrainsetFromFolder('/media/hdisk/liqiang/hyperSR/train/'+ opt.datasetName + '/' +  str(opt.upscale_factor) + '/')
    train_set = TrainsetFromFolder('/media/dy113/disk1/Project_xjr/dataset/hsisr_data/cave/cave_x4/train')
    train_loader = DataLoader(dataset=train_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=True)    
    # val_set = ValsetFromFolder('/media/hdisk/liqiang/hyperSR/test/' + opt.datasetName + '/' + str(opt.upscale_factor))
    val_set = TrainsetFromFolder('/media/dy113/disk1/Project_xjr/dataset/hsisr_data/cave/cave_x4/val')
    val_loader = DataLoader(dataset=val_set, num_workers=opt.threads, batch_size=opt.batchSize, shuffle=False)
      
    # Buliding model     
    model = LGFFMN()
    criterion1 = nn.L1Loss() 
    criterion2 = SAMLoss() 
    
    if opt.cuda:
        model = nn.DataParallel(model).cuda()
        criterion1 = criterion1.cuda()
        criterion2 = criterion2.cuda()
    else:
        model = model.cpu()   
    print('# parameters:', sum(param.numel() for param in model.parameters())) 
                   
    # Setting Optimizer
    optimizer = optim.Adam(model.parameters(),  lr=opt.lr,  betas=(0.9, 0.999), eps=1e-08)    

    # optionally resuming from a checkpoint
    if resume:
        if os.path.isfile(opt.resume):
            print("=> loading checkpoint '{}'".format(opt.resume))
            checkpoint = torch.load(opt.resume)         
            opt.start_epoch = checkpoint['epoch'] + 1 
            model.load_state_dict(checkpoint['model'])
            optimizer.load_state_dict(checkpoint['optimizer'])
        else:
            print("=> no checkpoint found at '{}'".format(opt.resume))       

    # Setting learning rate
    scheduler = MultiStepLR(optimizer, milestones=[35, 70, 105, 140, 175], gamma=0.5, last_epoch=-1) 
    # scheduler = MultiStepLR(optimizer, milestones=[50, 100, 150, 200], gamma=0.5, last_epoch = -1) 


    # Val_psnr = []
    # Training 
    for epoch in range(opt.start_epoch, opt.nEpochs + 1):
        # scheduler.step()
        print("Epoch = {}, lr = {}".format(epoch, optimizer.param_groups[0]["lr"])) 
        train(train_loader, optimizer, model, criterion1, criterion2, epoch) 
        scheduler.step() 
        # Val_psnr = val(val_loader, model, epoch)    
        val(val_loader, model, epoch)    

        if (epoch) % 5 == 0:        
            save_checkpoint(opt, epoch, model, optimizer)
        # EarlyStopping(Val_psnr)
        # if EarlyStopping.early_stop:
        #     save_checkpoint(opt, epoch, model, optimizer)
        #     print("Early stopping")
        #     break  

def train(train_loader, optimizer, model, criterion1, criterion2, epoch):

    model.train()   
      
    for iteration, batch in enumerate(train_loader, 1):
        # input, label = Variable(batch[0]),  Variable(batch[1], requires_grad=False)
        input, bicu, label = Variable(batch[0]),  Variable(batch[1]), Variable(batch[2], requires_grad=False)


        if opt.cuda:
            input = input.cuda()
            bicu = bicu.cuda()
            label = label.cuda()  

            # SR, localFeats = model(x, y, i)    
            SR = model(input, bicu)    

            # localFeats.detach_()
            # localFeats = localFeats.detach()
            # localFeats = Variable(localFeats.data, requires_grad=False)
            
            loss = criterion1(SR, label) + criterion2(SR, label)
            loss1 = criterion1(SR, label)
            loss2 = criterion2(SR, label)

            # loss = criterion1(SR, label) 


            optimizer.zero_grad()
            loss.backward()       
            optimizer.step()         
                                   
        if iteration % 50 == 0:
            # print("===> Epoch[{}]({}/{}): Loss: {:.10f}".format(epoch, iteration, len(train_loader), loss.data[0]))
            print("===> Epoch[{}]({}/{}): Loss: {:.10f} Loss1: {:.10f} Loss2: {:.10f}".format(epoch, iteration, len(train_loader), loss.item(), loss1.item(), loss2.item()))
            # print("===> Epoch[{}]({}/{}): Loss: {:.10f} ".format(epoch, iteration, len(train_loader), loss.item()))


        # if opt.show:
        #     writer.add_scalar('Train/Loss', loss.data[0], niter) 
        niter = epoch * len(train_loader) + iteration
        if niter % 500 == 0:
            # writer.add_scalar('Train/Loss', loss.data[0], niter)
            writer.add_scalar('Train/Loss', loss.item(), niter) 
        # writer.add_scalar('Trainepoch/Loss', loss.item(), epoch)  

def val(val_loader, model, epoch):	            

    model.eval()
    val_psnr = 0    
    
    with torch.no_grad():
        for iteration, batch in enumerate(val_loader, 1):
            # input, label = Variable(batch[0], volatile=True),  Variable(batch[1])
            input, bicu, label = Variable(batch[0]),  Variable(batch[1]), Variable(batch[2], requires_grad=False)
            SR = np.ones((label.shape[1], label.shape[2], label.shape[3])).astype(np.float32) 
        
            if opt.cuda:
                input = input.cuda()
                bicu = bicu.cuda()
    
                # output, localFeats = model(x, y, localFeats, i)    

                output = model(input, bicu)              
          
                SR = output.cpu().data[0].numpy()  
  	
            val_psnr += PSNR(SR, label.data[0].numpy()) 
        val_psnr = val_psnr / len(val_loader) 
        print("PSNR = {:.3f}".format(val_psnr))       
        if opt.show:
            writer.add_scalar('Val/PSNR',val_psnr, epoch) 
    # return val_psnr     
    
        
def save_checkpoint(args, epoch, model, optimizer):
    # model_out_path = "checkpoint/" + "{}_model_{}_epoch_{}.pth".format(opt.datasetName, opt.upscale_factor, epoch)
    checkpoint_model_dir = './checkpoints/' + args.model_title + '/' + args.datasetName + '/' + str(args.upscale_factor) + '/' + 'nfeats128nblock4sub8_34_batch3200/'
    if not os.path.exists(checkpoint_model_dir):
        os.makedirs(checkpoint_model_dir)

    ckpt_model_filename = args.datasetName + "_" + args.model_title + "_ckpt_epoch_" + str(epoch) + ".pth"
    ckpt_model_path = os.path.join(checkpoint_model_dir, ckpt_model_filename)
    state = {"epoch": epoch , "model": model.state_dict(), "optimizer":optimizer.state_dict()}
    # if not os.path.exists("checkpoint/"):
    #     os.makedirs("checkpoint/")     	
    # torch.save(state, model_out_path)
    torch.save(state, ckpt_model_path)
    print("Checkpoint saved to {}".format(ckpt_model_path))


if __name__ == "__main__":
    main()


=> Use GPU ID: '2,3'
# parameters: 7862856
=> loading checkpoint '/media/dy113/disk1/Project_xjr/hyperfusion/Mine/checkpoints/WTSR00/CAVE/4/nfeats128nblock4sub8_34_batch3200/CAVE_WTSR00_ckpt_epoch_85.pth'
Epoch = 86, lr = 5e-05
===> Epoch[86](50/248): Loss: 0.0106652901 Loss1: 0.0061360635 Loss2: 0.0045292270
===> Epoch[86](100/248): Loss: 0.0134341503 Loss1: 0.0090000145 Loss2: 0.0044341357
===> Epoch[86](150/248): Loss: 0.0119777005 Loss1: 0.0074883611 Loss2: 0.0044893394
===> Epoch[86](200/248): Loss: 0.0114008691 Loss1: 0.0071346615 Loss2: 0.0042662071
PSNR = 37.187
Epoch = 87, lr = 5e-05
===> Epoch[87](50/248): Loss: 0.0117827114 Loss1: 0.0073879845 Loss2: 0.0043947273
===> Epoch[87](100/248): Loss: 0.0149134099 Loss1: 0.0097671328 Loss2: 0.0051462776
===> Epoch[87](150/248): Loss: 0.0120050870 Loss1: 0.0073162615 Loss2: 0.0046888250
===> Epoch[87](200/248): Loss: 0.0133825857 Loss1: 0.0084159086 Loss2: 0.0049666776
PSNR = 37.210
Epoch = 88, lr = 5e-05
===> Epoch[88](50/248): Loss

In [4]:
def main():

    # input_path = '/media/hdisk/liqiang/hyperSR/test/' + opt.datasetName + '/' + str(opt.upscale_factor) + '/' 
    input_path = '/media/dy113/disk1/Project_xjr/dataset/hsisr_data/cave/cave_x4/test/'
    # out_path = '/media/hdisk/liqiang/hyperSR/result/' +  opt.datasetName + '/' + str(opt.upscale_factor) + '/' + opt.method + '/'
    out_path = '/media/dy113/disk1/Project_xjr/hyperfusion/Mine/result/' + opt.model_title + '/' + opt.datasetName + '/' + str(opt.upscale_factor)  + '/' + 'nfeats128nblock4sub8_34_batch3200/'

    if not os.path.exists(out_path):
        os.makedirs(out_path)      
    PSNRs = []
    SSIMs = []
    SAMs = []


    if not os.path.exists(out_path):
        os.makedirs(out_path)
                    
    if opt.cuda:
        print("=> use gpu id: '{}'".format(opt.gpus))
        os.environ["CUDA_VISIBLE_DEVICES"] = opt.gpus
        if not torch.cuda.is_available():
            raise Exception("No GPU found or Wrong gpu id, please run without --cuda")

    model = LGFFMN()

    if opt.cuda:
        model = nn.DataParallel(model).cuda()    
        
    checkpoint  = torch.load(opt.model_name)
    model.load_state_dict(checkpoint['model']) 
    model.eval()       
    
    images_name = [x for x in listdir(input_path) if is_image_file(x)]  

    with torch.no_grad():        
        T = 0        
        for index in range(len(images_name)):

            mat = scio.loadmat(input_path + images_name[index]) 
            hyperLR = mat['LR'].astype(np.float32).transpose(2,0,1)
            HR = mat['HR'].astype(np.float32).transpose(2,0,1)  

            bic = np.zeros(HR.shape, dtype=np.float32)  

            for i in range(bic.shape[0]):
                # bic[i,:,:] = imresize(LR[i,:,:], (bic.shape[1], bic.shape[2]), 'bicubic', mode='F')  
                # bic[i,:,:] = imresize(LR[i,:,:], (bic.shape[1], bic.shape[2]), 'bicubic', mode='F')  
                bic[i,:,:] = np.array(Image.fromarray(obj=hyperLR[i,:,:], mode='F').resize(size=(bic.shape[1], bic.shape[2]), resample=Image.BICUBIC))        

            bicu = Variable(torch.from_numpy(bic).float(), volatile=True).contiguous().view(1, -1, bic.shape[1], bic.shape[2])                  	            	      	    	        	


            input = Variable(torch.from_numpy(hyperLR).float(), volatile=True).contiguous().view(1, -1, hyperLR.shape[1], hyperLR.shape[2])      
            HR = np.array(HR).astype(np.float32) 
         
            if opt.cuda:
                input = input.cuda() 
                bicu =bicu.cuda() 
           
            # localFeats = []
            start = time.time()
     
            output = model(input, bicu)                                 
                             
            SR = output.cpu().data[0].numpy().astype(np.float32)  
                
            end = time.time()   
            print (end-start)
            T = T + (end - start)
                   
            SR[SR<0] = 0             
            SR[SR>1.] = 1.
            psnr = PSNR(SR, HR)
            ssim = SSIM(SR, HR)
            sam = SAM(SR, HR)
        
            PSNRs.append(psnr)
            SSIMs.append(ssim)
            SAMs.append(sam)
        
            SR = SR.transpose(1,2,0)   
            HR = HR.transpose(1,2,0)  
        
	                    
            scio.savemat(out_path + images_name[index], {'HR': HR, 'SR':SR})  
            print("===The {}-th picture=====PSNR:{:.3f}=====SSIM:{:.4f}=====SAM:{:.3f}====Name:{}".format(index+1,  psnr, ssim, sam, images_name[index]))                 
        print("=====averPSNR:{:.3f}=====averSSIM:{:.4f}=====averSAM:{:.3f}".format(np.mean(PSNRs), np.mean(SSIMs), np.mean(SAMs))) 
        print (T/len(images_name))

        
if __name__ == "__main__":
    main()

=> use gpu id: '2,3'


/home/dy113/.local/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/dy113/.local/lib/python3.6/site-packages/ipykernel_launcher.py:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


0.1512317657470703
===The 1-th picture=====PSNR:35.088=====SSIM:0.9476=====SAM:4.153====Name:thread_spools_ms.mat
0.14707088470458984
===The 2-th picture=====PSNR:43.116=====SSIM:0.9906=====SAM:2.137====Name:real_and_fake_peppers_ms.mat
0.14441347122192383
===The 3-th picture=====PSNR:45.755=====SSIM:0.9941=====SAM:2.461====Name:real_and_fake_apples_ms.mat
0.14646410942077637
===The 4-th picture=====PSNR:41.272=====SSIM:0.9820=====SAM:2.524====Name:stuffed_toys_ms.mat
0.1463935375213623
===The 5-th picture=====PSNR:39.838=====SSIM:0.9845=====SAM:1.421====Name:sponges_ms.mat
0.14647412300109863
===The 6-th picture=====PSNR:42.689=====SSIM:0.9797=====SAM:5.835====Name:superballs_ms.mat
=====averPSNR:41.293=====averSSIM:0.9798=====averSAM:3.089
0.14700798193613687
